Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))  / predictions.shape[0])


---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### Logistic Regression

In [5]:
# I copied some code from assignment 2 that complished yesterday

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
lambda_term = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size ** 2, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) +
                          lambda_term * tf.nn.l2_loss(weights) +
                          lambda_term * tf.nn.l2_loss(biases))
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    # Set small learning rate 
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    # train_prediction = tf.nn.softmax(logits) # logits is equal to `tf.matmul(tf_train_dataset, weights) + biases`
    train_prediction = tf.nn.softmax(tf.matmul(tf_train_dataset, weights) + biases)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

run this compute and iterate

In [6]:
# I copied some code from assignment 2 that complished yesterday

num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, and random.
    # tf.global_variables_initializer().run()
    tf.initialize_all_variables().run() 
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss at step 0: 50.904644
Training accuracy: 7.4%
Validation accuracy: 8.7%
Loss at step 100: 11.921806
Training accuracy: 74.6%
Validation accuracy: 72.0%
Loss at step 200: 4.500904
Training accuracy: 79.7%
Validation accuracy: 76.9%
Loss at step 300: 1.977441
Training accuracy: 82.8%
Validation accuracy: 80.0%
Loss at step 400: 1.121218
Training accuracy: 84.3%
Validation accuracy: 81.3%
Loss at step 500: 0.827709
Training accuracy: 85.0%
Validation accuracy: 81.7%
Loss at step 600: 0.725956
Training accuracy: 85.1%
Validation accuracy: 81.8%
Loss at step 700: 0.690377
Training accuracy: 85.2%
Validation accuracy: 81.8%
Loss at step 800: 0.677858
Training accuracy: 85.3%
Validation accuracy: 81.8%
Test accuracy: 88.6%


#### Full Connected Neural Network with one hidden-layer

In [7]:
# ReLU neuron

# param
training_epochs = 30
batch_size = 521
display_step = 1
n_input = 784 # img shape: 28*28
n_classes = 10 # MNIST total classes (0-9 digits)

# hyper-parameter
n_hidden_1 = 256 
learning_rate = 0.05
lambda_term = 0.01


graph = tf.Graph()
with graph.as_default():
    # init weights
    weights_hiden =  tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=1.0/np.sqrt(n_input)))
    weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_classes], stddev=1.0/np.sqrt(n_hidden_1)))
    
    biases_hidden = tf.Variable(tf.random_normal([n_hidden_1]))
    biases_out = tf.Variable(tf.random_normal([n_classes]))
    
    x = tf.placeholder("float", [None, n_input])
    y = tf.placeholder("float", [None, n_classes])
    
    def model(x, weights_hiden, weights_out, biases_hidden, biases_out):
        # hidden layer with RELU activation
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights_hiden), biases_hidden))
        # output layer with linear activation
        out_layer = tf.matmul(layer_1, weights_out) + biases_out
        return out_layer
    
    # Construct model
    pred = model(x, weights_hiden, weights_out, biases_hidden, biases_out)
    
    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y) +
                          lambda_term * tf.nn.l2_loss(weights_hiden) + 
                          lambda_term * tf.nn.l2_loss(weights_out) +
                          lambda_term * tf.nn.l2_loss(biases_hidden) + 
                          lambda_term * tf.nn.l2_loss(biases_out))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)


run the graph

In [8]:
# Launch the graph
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print('Initialized')
    # Training cycle
    for epoch in range(training_epochs):
        offset = (epoch * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_x = train_dataset[offset:(offset + batch_size), :]
        batch_y = train_labels[offset:(offset + batch_size), :]
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
        # Compute average loss
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Test data accuracy:", accuracy.eval({x: test_dataset, y: test_labels}))
    print("Valid data accuracy:", accuracy.eval({x: valid_dataset, y: valid_labels}))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Epoch: 0001 cost= 5.355447769
Epoch: 0002 cost= 4.941251755
Epoch: 0003 cost= 4.762169838
Epoch: 0004 cost= 4.625680923
Epoch: 0005 cost= 4.588827133
Epoch: 0006 cost= 4.479930401
Epoch: 0007 cost= 4.449034691
Epoch: 0008 cost= 4.396099091
Epoch: 0009 cost= 4.367498398
Epoch: 0010 cost= 4.298876286
Epoch: 0011 cost= 4.224603176
Epoch: 0012 cost= 4.145647049
Epoch: 0013 cost= 4.162161350
Epoch: 0014 cost= 4.128488541
Epoch: 0015 cost= 4.086365223
Epoch: 0016 cost= 4.037142277
Epoch: 0017 cost= 4.013607979
Epoch: 0018 cost= 3.960484505
Epoch: 0019 cost= 3.921268940
Epoch: 0020 cost= 3.902770758
Epoch: 0021 cost= 3.899853706
Epoch: 0022 cost= 3.918957949
Epoch: 0023 cost= 3.811633110
Epoch: 0024 cost= 3.782674551
Epoch: 0025 cost= 3.802799940
Epoch: 0026 cost= 3.741531372
Epoch: 0027 cost= 3.725215912
Epoch: 0028 cost= 3.685925484
Epoch: 0029 cost= 3.651138544
Epoch: 0030 cost= 3.655175209
Optimization F

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

#### Here just show the result of Full Connected Neural Network with one-hidden layer with ReLU neuron

In [9]:
# I just copy all the code above

# ReLU neuron

# param
training_epochs = 30
batch_size = 1 #  Here
display_step = 1
n_input = 784 # img shape: 28*28
n_classes = 10 # MNIST total classes (0-9 digits)

# hyper-parameter
n_hidden_1 = 256 
learning_rate = 0.05
lambda_term = 0.01


graph = tf.Graph()
with graph.as_default():
    # init weights
    weights_hiden =  tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=1.0/np.sqrt(n_input)))
    weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_classes], stddev=1.0/np.sqrt(n_hidden_1)))
    
    biases_hidden = tf.Variable(tf.random_normal([n_hidden_1]))
    biases_out = tf.Variable(tf.random_normal([n_classes]))
    
    x = tf.placeholder("float", [None, n_input])
    y = tf.placeholder("float", [None, n_classes])
    
    def model(x, weights_hiden, weights_out, biases_hidden, biases_out):
        # hidden layer with RELU activation
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights_hiden), biases_hidden))
        # output layer with linear activation
        out_layer = tf.matmul(layer_1, weights_out) + biases_out
        return out_layer
    
    # Construct model
    pred = model(x, weights_hiden, weights_out, biases_hidden, biases_out)
    
    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y) +
                          lambda_term * tf.nn.l2_loss(weights_hiden) + 
                          lambda_term * tf.nn.l2_loss(weights_out) +
                          lambda_term * tf.nn.l2_loss(biases_hidden) + 
                          lambda_term * tf.nn.l2_loss(biases_out))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
    

In [10]:
# Launch the graph
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print('Initialized')
    # Training cycle
    for epoch in range(training_epochs):
        offset = (epoch * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_x = train_dataset[offset:(offset + batch_size), :]
        batch_y = train_labels[offset:(offset + batch_size), :]
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
        # Compute average loss
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Test data accuracy:", accuracy.eval({x: test_dataset, y: test_labels}))
    print("Valid data accuracy:", accuracy.eval({x: valid_dataset, y: valid_labels}))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Epoch: 0001 cost= 3.243125677
Epoch: 0002 cost= 7.185295582
Epoch: 0003 cost= 9.017312050
Epoch: 0004 cost= 10.810412407
Epoch: 0005 cost= 9.161843300
Epoch: 0006 cost= 7.768702507
Epoch: 0007 cost= 7.393570423
Epoch: 0008 cost= 15.218988419
Epoch: 0009 cost= 9.775375366
Epoch: 0010 cost= 5.823899746
Epoch: 0011 cost= 8.362657547
Epoch: 0012 cost= 4.737264633
Epoch: 0013 cost= 10.912469864
Epoch: 0014 cost= 7.934610367
Epoch: 0015 cost= 11.137799263
Epoch: 0016 cost= 7.474731922
Epoch: 0017 cost= 12.397913933
Epoch: 0018 cost= 6.339158535
Epoch: 0019 cost= 5.111407280
Epoch: 0020 cost= 7.106061935
Epoch: 0021 cost= 3.141166210
Epoch: 0022 cost= 5.446547508
Epoch: 0023 cost= 6.799573421
Epoch: 0024 cost= 8.185979843
Epoch: 0025 cost= 2.589447498
Epoch: 0026 cost= 2.611497879
Epoch: 0027 cost= 2.677704334
Epoch: 0028 cost= 4.996290684
Epoch: 0029 cost= 2.566218376
Epoch: 0030 cost= 2.829747677
Optimizat

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
# I just copy the code above
# ReLU neuron

import random


# param
training_epochs = 10000
batch_size = 128
display_step = 1000
n_input = 784 # img shape: 28*28
n_classes = 10 # MNIST total classes (0-9 digits)

# hyper-parameter
n_hidden_1 = 1024
learning_rate = 0.05
lambda_term = 0.01


graph = tf.Graph()
with graph.as_default():
    # init weights
    weights_hiden =  tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=1.0/np.sqrt(n_input)))
    weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_classes], stddev=1.0/np.sqrt(n_hidden_1)))
    
    biases_hidden = tf.Variable(tf.random_normal([n_hidden_1]))
    biases_out = tf.Variable(tf.random_normal([n_classes]))
    
    x = tf.placeholder("float", [None, n_input])
    y = tf.placeholder("float", [None, n_classes])
    keep = tf.placeholder(tf.float32)
    
    
    def model(x, weights_hiden, weights_out, biases_hidden, biases_out, keep):
        # hidden layer with RELU activation
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights_hiden), biases_hidden))
        # apply DropOut to hidden layer
        drop_out = tf.nn.dropout(layer_1, keep)
        # output layer with linear activation
        out_layer = tf.matmul(drop_out, weights_out) + biases_out
        return out_layer
    
    def forward_propagate(x, weights_hiden, weights_out, biases_hidden, biases_out):
        # hidden layer with RELU activation
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights_hiden), biases_hidden))
        # output layer with linear activation
        out_layer = tf.matmul(layer_1, weights_out) + biases_out
        return out_layer
    
    # Construct model
    pred = model(x, weights_hiden, weights_out, biases_hidden, biases_out, keep)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y) +
                          lambda_term * tf.nn.l2_loss(weights_hiden) + 
                          lambda_term * tf.nn.l2_loss(weights_out) +
                          lambda_term * tf.nn.l2_loss(biases_hidden) + 
                          lambda_term * tf.nn.l2_loss(biases_out))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    test_pred = forward_propagate(x, weights_hiden, weights_out, biases_hidden, biases_out)

    
# run the graph
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print('Initialized')
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        offset = (epoch * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_x = train_dataset[offset:(offset + batch_size), :]
        batch_y = train_labels[offset:(offset + batch_size), :]
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y, keep: 0.5})
        # Compute average loss
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(test_pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Test data accuracy:", accuracy.eval({x: test_dataset, y: test_labels}))
    print("Valid data accuracy:", accuracy.eval({x: valid_dataset, y: valid_labels}))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Epoch: 0001 cost= 13.494695663
Epoch: 1001 cost= 4.388029575
Epoch: 2001 cost= 2.001630068
Epoch: 3001 cost= 1.129750848
Epoch: 4001 cost= 0.982329130
Epoch: 5001 cost= 0.702153683
Epoch: 6001 cost= 0.747316241
Epoch: 7001 cost= 0.834857345
Epoch: 8001 cost= 0.677043140
Epoch: 9001 cost= 0.681712925
Optimization Finished!
Test data accuracy: 0.9081
Valid data accuracy: 0.8451


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [13]:
# improve the accuracy with multi-layer neural network
# with l2-regularization and DropOut 
# Here define 2 hidden-layers Full Connected Neural Network.

# I just copy some code above

import random

# param
training_epochs = 30000
batch_size = 128
display_step = 1000
n_input = 784 # img shape: 28*28
n_classes = 10 # MNIST total classes (0-9 digits)

# hyper-parameter
n_hidden_1 = 1024
n_hidden_2 = 1024
n_hidden_3 = 1024
learning_rate = 0.005

def accuracy_func(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))  / predictions.shape[0])


graph = tf.Graph()
with graph.as_default():
    # init weights
    weights_hidden_1 =  tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=1.0/np.sqrt(n_input)))
    weights_hidden_2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=1.0/np.sqrt(n_hidden_1)))
    weights_hidden_3 = tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], stddev=1.0/np.sqrt(n_hidden_2)))    
    weights_out = tf.Variable(tf.random_normal([n_hidden_3, n_classes], stddev=1.0/np.sqrt(n_hidden_3)))
    # init biases
    # biases_hidden_1 = tf.Variable(tf.random_normal([n_hidden_1]))
    biases_hidden_1 = tf.Variable(tf.zeros([n_hidden_1]))
    # biases_hidden_2 = tf.Variable(tf.random_normal([n_hidden_2]))
    biases_hidden_2 = tf.Variable(tf.zeros([n_hidden_2]))
    # biases_hidden_3 = tf.Variable(tf.random_normal([n_hidden_3]))
    biases_hidden_3 = tf.Variable(tf.zeros([n_hidden_3]))
    # biases_out = tf.Variable(tf.random_normal([n_classes]))
    biases_out = tf.Variable(tf.zeros([n_classes]))
    
    # make a dict
    weights = {'1': weights_hidden_1, 
                       '2': weights_hidden_2,  
                       '3':weights_hidden_3, 
                       'out': weights_out}
    biases = {'1': biases_hidden_1, 
                     '2': biases_hidden_2, 
                     '3':biases_hidden_3, 
                     'out': biases_out}
    
    x = tf.placeholder("float", [None, n_input])
    y = tf.placeholder("float", [None, n_classes])
    keep_prob = tf.placeholder(tf.float32)
    
    # in fact, this is a forwad propagation function
    def forward_propagate(x, weights, bises, keep_prob):
        # Hidden layer 1  with RELU activation
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['1']), biases['1']))
        # apply DropOut to hidden layer
        drop_out_1 = tf.nn.dropout(layer_1, keep_prob)
        # Hidden layer 2 with RELU activation
        layer_2 = tf.nn.relu(tf.add(tf.matmul(drop_out_1, weights['2']), biases['2']))
        # apply DropOut to hidden layer
        drop_out_2 = tf.nn.dropout(layer_2, keep_prob)
        # Hidden layer 3 with RELU activation
        layer_3 = tf.nn.relu(tf.add(tf.matmul(drop_out_2, weights['3']), biases['3']))
        # apply DropOut to hidden layer
        drop_out_3 = tf.nn.dropout(layer_3, keep_prob)
        # Output layer with linear activation
        out_layer = tf.matmul(drop_out_3, weights['out']) + biases['out']
        return out_layer


    def predict(x, weights_hiden, weights_out, biases_hidden, biases_out):
        # Hidden layer 1  with RELU activation
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['1']), biases['1']))
        # Hidden layer 2 with RELU activation
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['2']), biases['2']))
        # Hidden layer 3 with RELU activation
        layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['3']), biases['3']))
        # Output layer with linear activation
        predict = tf.nn.softmax(tf.matmul(layer_3, weights['out']) + biases['out'])
        return predict
    
    # Construct model
    logits = forward_propagate(x, weights, biases, keep_prob)
    
    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_rate=0.25,  global_step=global_step, decay_steps=5000, decay_rate=0.90, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost, global_step=global_step)
    
    pred = tf.nn.softmax(logits)
    test_pred = predict(tf.constant(test_dataset), weights_hiden, weights_out, biases_hidden, biases_out)
    valid_pred = predict(tf.constant(valid_dataset), weights_hiden, weights_out, biases_hidden, biases_out)


# run the graph
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    # Training cycle
    for epoch in range(training_epochs):
        offset = (epoch * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_x = train_dataset[offset:(offset + batch_size), :]
        batch_y = train_labels[offset:(offset + batch_size), :]
        # Run optimization op (backprop) and cost op (to get loss value)
        _, res_cost, prediction = sess.run([optimizer, cost, pred], feed_dict={x: batch_x, y: batch_y, keep_prob: 0.9})
        # Compute average loss
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(res_cost), ' train accuracy:{:.2f}% '.format(accuracy_func(prediction, batch_y)), 
                  ' test accuracy:{:.2f}%'.format(accuracy_func(test_pred.eval(), test_labels)), 
                  ' valid accuracy:{:.2f}% '.format(accuracy_func(valid_pred.eval(), valid_labels)))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(test_pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Test data accuracy:{:.2f}%".format(accuracy_func(test_pred.eval(), test_labels)))
    print("Valid data accuracy:{:.2f}%".format(accuracy_func(valid_pred.eval() ,valid_labels)))


Initialized
Epoch: 0001 cost= 2.331088543  train accuracy:10.94%   test accuracy:30.40%  valid accuracy:28.14% 
Epoch: 1001 cost= 0.375692695  train accuracy:88.28%   test accuracy:92.35%  valid accuracy:86.29% 
Epoch: 2001 cost= 0.264163882  train accuracy:90.62%   test accuracy:94.10%  valid accuracy:88.56% 
Epoch: 3001 cost= 0.213865727  train accuracy:92.97%   test accuracy:94.82%  valid accuracy:89.58% 
Epoch: 4001 cost= 0.333704442  train accuracy:89.06%   test accuracy:95.16%  valid accuracy:89.86% 
Epoch: 5001 cost= 0.185625821  train accuracy:92.97%   test accuracy:95.38%  valid accuracy:89.92% 
Epoch: 6001 cost= 0.282529622  train accuracy:92.19%   test accuracy:95.78%  valid accuracy:90.39% 
Epoch: 7001 cost= 0.314455748  train accuracy:90.62%   test accuracy:95.71%  valid accuracy:90.56% 
Epoch: 8001 cost= 0.209148213  train accuracy:95.31%   test accuracy:95.66%  valid accuracy:90.20% 
Epoch: 9001 cost= 0.178473949  train accuracy:94.53%   test accuracy:95.93%  valid accur

In [14]:
batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 1024
num_hidden_nodes3 = 1024
keep_prob = 0.9


graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights1 = tf.Variable(tf.truncated_normal(
            [image_size * image_size, num_hidden_nodes1], stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
    weights2 = tf.Variable(tf.truncated_normal(
            [num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
    biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
    weights3 = tf.Variable(tf.truncated_normal(
            [num_hidden_nodes2, num_hidden_nodes3], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
    biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]))
    weights4 = tf.Variable(tf.truncated_normal(
            [num_hidden_nodes3, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes3)))
    biases4 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    def forward_propagate():
        lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
        drop1 = tf.nn.dropout(lay1_train, keep_prob)
        lay2_train = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
        drop2 = tf.nn.dropout(lay2_train, keep_prob)
        lay3_train = tf.nn.relu(tf.matmul(drop2, weights3) + biases3)
        drop3 = tf.nn.dropout(lay3_train, keep_prob)
        logits = tf.matmul(drop3, weights4) + biases4
        return logits
    
    # Predictions for the training, validation, and test data.
    def predict(dataset):
        lay1_valid = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
        lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
        lay3_valid = tf.nn.relu(tf.matmul(lay2_valid, weights3) + biases3)
        prediction = tf.nn.softmax(tf.matmul(lay3_valid, weights4) + biases4)
        return prediction

    logits = forward_propagate()
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(learning_rate=0.5, global_step=global_step, decay_steps=5000, decay_rate=0.80, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = predict(tf_valid_dataset)
    test_prediction = predict(tf_test_dataset)

In [15]:
def accuracy_func(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))  / predictions.shape[0])

num_epoch = 30001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for epoch in range(num_epoch):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (epoch * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # feed data
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, c, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (epoch % 1000 == 0):
            print("Minibatch loss at epoch %d: %f" % (epoch, c))
            print("Minibatch accuracy: %.1f%%" % accuracy_func(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy_func(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy_func(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at epoch 0: 2.409556
Minibatch accuracy: 9.4%
Validation accuracy: 17.0%
Minibatch loss at epoch 1000: 0.402116
Minibatch accuracy: 88.3%
Validation accuracy: 86.5%
Minibatch loss at epoch 2000: 0.259861
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at epoch 3000: 0.205248
Minibatch accuracy: 93.0%
Validation accuracy: 89.3%
Minibatch loss at epoch 4000: 0.303189
Minibatch accuracy: 90.6%
Validation accuracy: 90.0%
Minibatch loss at epoch 5000: 0.125371
Minibatch accuracy: 96.1%
Validation accuracy: 89.5%
Minibatch loss at epoch 6000: 0.241323
Minibatch accuracy: 93.0%
Validation accuracy: 90.3%
Minibatch loss at epoch 7000: 0.260685
Minibatch accuracy: 90.6%
Validation accuracy: 90.6%
Minibatch loss at epoch 8000: 0.166088
Minibatch accuracy: 94.5%
Validation accuracy: 90.4%
Minibatch loss at epoch 9000: 0.218168
Minibatch accuracy: 94.5%
Validation accuracy: 90.5